<a href="https://colab.research.google.com/github/37stu37/rnc2/blob/master/temporal_network_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import sys
import os
from pathlib import Path
import networkx as nx
from numba import jit
from dask.distributed import Client
pd.options.mode.chained_assignment = None  # default='warn'

client = Client(processes=False)
client

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/distributed/bokeh/core.py:57: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 99] Cannot assign requested address
  warnings.warn('\n' + msg)
/usr/local/lib/python3.6/dist-packages/distributed/deploy/local.py:197: UserWarning: 
Could not launch service 'bokeh' on port 8787. Got the following message:

[Errno 99] Cannot assign requested address
  self.scheduler.start(scheduler_address)


Client Scheduler: inproc://172.28.0.2/126/1,Cluster Workers: 1 Cores: 2 Memory: 13.65 GB




---


**Tentative code for rnc2 temporal network**

@time

Conditions -> Boolean mask -> Contacts -> Edgelist


---



In [0]:
# import data
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/Academia/01_Publications/00_Alex/005_RNC2')
edge_file = folder / 'data' / 'Copy of edge_data.parquet'
wind_file = folder / 'data' / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

**Conditions**


---



In [0]:
def wind_scenario(wind_data):  
    i = np.random.randint(0, wind_data.values.shape[0])
    w = wind_data.values[i, 2]
    dist = wind_data.values[i, 1]
    b = wind_data.values[i, 3]
    bear_max = b + 45  # wind direction
    bear_min = b - 45
    if b == 360:
        bear_max = 45
    if b <= 0:  # should not be necessary
        bear_min = 0
    if b == 999:
        bear_max = 999
        bear_min = 0
    return bear_max, bear_min, dist # wind characteristics, bearing and distance

# ignition ?
def mask_ignition_rng(e):
    rng = np.random.uniform(0, 1, size=e.values.shape[0])
    mask = rng < e.IgnProb_bl.values
    return mask*1

# wind direction
def mask_wind_direction(e, w_bear_max, w_bear_min):
    mask = (e.bearing.values < w_bear_max) & (e.bearing.values > w_bear_min) 
    return mask*1

# wind speed
def mask_wind_speed(e, w_dist):
    mask = e.distance.values < w_dist
    return mask*1

# not burnt in previous fires ?
def mask_burnt(t, e, c):
    if t != 1:
      previouslyActivated = c[:,:t-1].sum(axis=1)
    else:
      previouslyActivated = c[:,0]
    mask = np.where(previouslyActivated > 0, 1, 0)
    # mask = mask == 0 # == 0 means that no fire previously occured
    return mask*1

# target -> source ?
def mask_target_to_source(t, e, c):
    previousTargetAtTime = e.target[c[:, t-1] == 1]
    mask = np.in1d(e.source.values, previousTargetAtTime.values)
    return mask*1

**Main**


---



In [0]:
# %%time
# for scenario in range(100):
    # time = 0 
    # # initial conditions
    # m_ignition_rng = mask_ignition_rng(edgelist)
    # print("{} ignitions of {} total ~ {} % of buildings".format(m_ignition_rng.sum(axis=0), edgelist.shape[0], 
    #                                                     (m_ignition_rng.sum(axis=0) / edgelist.shape[0])* 100))
    # ActTime = np.zeros((m_ignition_rng.shape[0],1))
    # ActTime[:, 0] = m_ignition_rng
    # # wind conditions and mask
    # w_direction_max, w_direction_min, w_distance = wind_scenario(wind_data)
    # m_wind_direction = mask_wind_direction(edgelist, w_direction_max, w_direction_min)
    # m_wind_speed = mask_wind_speed(edgelist, w_distance)
    # m_wind = m_wind_direction & m_wind_speed
    # # print("wind_direction : {}, wind_speed : {}, wind : {}".format(m_wind_direction[:5], m_wind_speed[:5], m_wind[:5]))
    # while ActTime[:,-1].sum(axis=0) != 0:
    #     time += 1
    #     print('{} still burning @ time {} ########'.format(int(ActTime[:,-1].sum(axis=0)), time))
    #     # fire mask
    #     m_targetSource = mask_target_to_source(time, edgelist, ActTime)
    #     print("{} target to source".format(m_targetSource.sum(axis=0)))
    #     m_burnt = mask_burnt(time, edgelist, ActTime)
    #     print("{} burnt".format(m_burnt.sum(axis=0)))
    #     m_fire = m_burnt & m_targetSource
    #     print("{} fire".format(m_fire.sum(axis=0)))
    #     # cumulated mask
    #     m_propagated= m_wind & m_fire
    #     print("{} propagated".format(m_propagated.sum(axis=0)))
    #     # append the activated layers to ActTime
    #     ActTime = np.c_[ActTime, m_propagated]
    #     print("ActTime : {}".format(ActTime.shape))
    #     # get the active edges
    #     ActEdges = edgelist[ActTime[:,time] == 1]
    #     print("ActEdges : {}".format(ActEdges.shape))

In [0]:
def wind_scenario(t, wind_data):
    if t == 0:
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance
    else:
      return

def wind_mask(e, w_bearing_max, w_bearing_min, w_distance):
    mask1 = (e.bearing.values < w_bearing_max) & (e.bearing.values > w_bearing_min)
    mask2 = e.distance.values < w_distance
    mask = mask1 & mask2
    return mask*1

def ignition_mask(t, e, c):
    # initial ignition
    if t == 0:
      rng = np.random.uniform(0, 1, size=e.values.shape[0])
      mask = rng < e.IgnProb_bl.values
      return mask*1
    else:
    # propagation
      mask_previous_time = c.iloc[:, t-1] == 1
      previousTarget = e.target.iloc[mask_previous_time]
      mask = np.in1d(e.source.values, previousTarget.values)
      return mask*1

def burnt_mask(t, e, c):
    if t > 1:
      previouslyActivated = c.iloc[:,:t-1]
      previouslyActivated_sum = previouslyActivated.sum(axis=1)
      mask = np.where(previouslyActivated_sum> 0, 1, 0)
    else:
      mask = c.iloc[:,0].values
    return mask*1

In [31]:
n=1
for scenario in range(n):
    time = 0 
    CoTime = pd.DataFrame(np.ones((len(edgelist), 1)))
    bear_max, bear_min, distance = wind_scenario(time, wind_data)
    while CoTime.iloc[:,-1].sum(axis=0) > 0:
        print("scenario : {} time : {}".format(scenario, time))
        # ignititon and propagation
        ignition_m = ignition_mask(time, edgelist, CoTime)
        print(type(ignition_m))
        # targets under the wind
        wind_m = wind_mask(edgelist, bear_max, bear_min, distance)
        print(type(wind_m))
        # not already burnt
        burnt_m = burnt_mask(time, edgelist, CoTime)
        print(type(burnt_m))
        # resulting mask
        mask == (ignition_m & wind_m & burnt_m)
        mask = np.where(mask> 0, 1, 0)
        ActiveEdges = edgelist[mask]
        print("number of active edges : {}".format(len(ActiveEdges)))
        time += 1

scenario : 0 time : 0
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


TypeError: ignored